In [1]:
import zipfile
import pathlib
import string
import random
from collections import Counter
from typing import List

from IPython.display import HTML, display
from tabulate import tabulate 

In [2]:
%load_ext autoreload

%autoreload 2

from utils import read_text, preprocess_text, LetterPermutation, get_emoji_vocab
from language_model import count_ngram, NGramStat
from simple_decoder import beam_search, build_graph, decode_ngrams, encode_text, decode_text
from mcmc_decoding import mcmc_decryption
from constants import RUSSIAN_LETTERS

In [3]:
path_to_zip = pathlib.Path("data", "corpora.zip")

In [4]:
texts = read_text(path_to_zip, ignore_files=["WarAndPeaceEng.txt"])

In [5]:
for text in texts:
    texts[text] = preprocess_text(texts[text])

In [6]:
n_gram = 1

In [7]:
source_freq = Counter()
for name in texts:
    if name == "AnnaKarenina.txt":
        big_text = texts[name]
    else:
        source_freq.update(count_ngram(texts[name], n_gram=n_gram))

In [8]:
unigram_stat = NGramStat(source_freq)

In [9]:
original_vocab = set(RUSSIAN_LETTERS)
original_vocab.add(" ")
emodji_vocab = list(get_emoji_vocab())

symmetric_key = LetterPermutation(original_vocab, set(random.sample(emodji_vocab, k=len(original_vocab))))

permutations = list(range(len(symmetric_key)))
random.shuffle(permutations)
symmetric_key.permute(permutations)
del permutations

# Частотное декодирование на основе частот отдельных симовлов

In [10]:
test_text = "Вронский слушал с удовольствием этот веселый лепет хорошенькой женщины, поддакивая ей, давал полушутливые советы и вообще тотчас же принял свой привычный тон обращения с этого рода женщинами. В его петербургском мире все люди разделялись на два совершенно противоположные сорта. Один низший сорт: пошлые, глупые и, главное, смешные люди, которые веруют в то, что одному мужу надо жить с одною женой, с которою он обвенчан, что девушке надо быть невинною, женщине стыдливою, мужчине мужественным, воздержным и твердым, что надо воспитывать детей, зарабатывать свой хлеб, платить долги, — и разные тому подобные глупости. Это был сорт людей старомодных и смешных. Но был другой сорт людей, настоящих, к которому они все принадлежали, в котором надо быть, главное, элегантным, красивым, великодушным, смелым, веселым, отдаваться всякой страсти не краснея и над всем остальным смеяться."

In [11]:
encoded_text, original_processed = encode_text(test_text, symmetric_key, n_gram=n_gram)

Encoded text length: 848


In [12]:
encoded_text

'🌙🎉🏯👤😜🍊🕖👱🌺😜👞💡📂📹👞🌺😜🌺💡🔝🏯🌙🏯👞💷😜📭🌙🕖🐱🔄🌺🍭📭🏯📭🌺🌙🐱😜🐱👞💑👱🌺👞🐱🎿🐱📭🌺💟🏯🎉🏯📂🐱👤💷🍊🏯👱🌺🌼🐱👤💖🕖👤💑🌺🎿🏯🔝🔝📹🍊🕖🌙📹🙆🌺🐱👱🌺🔝📹🌙📹👞🌺🎿🏯👞💡📂💡📭👞🕖🌙💑🐱🌺😜🏯🌙🐱📭💑🌺🕖🌺🌙🏯🏯💻💖🐱🌺📭🏯📭👨📹😜🌺🌼🐱🌺🎿🎉🕖👤🙆👞🌺😜🌙🏯👱🌺🎿🎉🕖🌙💑👨👤💑👱🌺📭🏯👤🌺🏯💻🎉📹💖🐱👤🕖🙆🌺😜🌺🍭📭🏯😡🏯🌺🎉🏯🔝📹🌺🌼🐱👤💖🕖👤📹🔄🕖🌺🌙🌺🐱😡🏯🌺🎿🐱📭🐱🎉💻💡🎉😡😜🍊🏯🔄🌺🔄🕖🎉🐱🌺🌙😜🐱🌺👞🔲🔝🕖🌺🎉📹🔫🔝🐱👞🙆👞🕖😜💷🌺👤📹🌺🔝🌙📹🌺😜🏯🌙🐱🎉📂🐱👤👤🏯🌺🎿🎉🏯📭🕖🌙🏯🎿🏯👞🏯🌼👤💑🐱🌺😜🏯🎉📭📹🌺🏯🔝🕖👤🌺👤🕖🔫📂🕖👱🌺😜🏯🎉📭🌺🎿🏯📂👞💑🐱🌺😡👞💡🎿💑🐱🌺🕖🌺😡👞📹🌙👤🏯🐱🌺😜🔄🐱📂👤💑🐱🌺👞🔲🔝🕖🌺🍊🏯📭🏯🎉💑🐱🌺🌙🐱🎉💡🔲📭🌺🌙🌺📭🏯🌺👨📭🏯🌺🏯🔝👤🏯🔄💡🌺🔄💡🌼💡🌺👤📹🔝🏯🌺🌼🕖📭💷🌺😜🌺🏯🔝👤🏯🔲🌺🌼🐱👤🏯👱🌺😜🌺🍊🏯📭🏯🎉🏯🔲🌺🏯👤🌺🏯💻🌙🐱👤👨📹👤🌺👨📭🏯🌺🔝🐱🌙💡📂🍊🐱🌺👤📹🔝🏯🌺💻💑📭💷🌺👤🐱🌙🕖👤👤🏯🔲🌺🌼🐱👤💖🕖👤🐱🌺😜📭💑🔝👞🕖🌙🏯🔲🌺🔄💡🌼👨🕖👤🐱🌺🔄💡🌼🐱😜📭🌙🐱👤👤💑🔄🌺🌙🏯🔫🔝🐱🎉🌼👤💑🔄🌺🕖🌺📭🌙🐱🎉🔝💑🔄🌺👨📭🏯🌺👤📹🔝🏯🌺🌙🏯😜🎿🕖📭💑🌙📹📭💷🌺🔝🐱📭🐱👱🌺🔫📹🎉📹💻📹📭💑🌙📹📭💷🌺😜🌙🏯👱🌺💟👞🐱💻🌺🎿👞📹📭🕖📭💷🌺🔝🏯👞😡🕖🌺🕖🌺🎉📹🔫👤💑🐱🌺📭🏯🔄💡🌺🎿🏯🔝🏯💻👤💑🐱🌺😡👞💡🎿🏯😜📭🕖🌺🍭📭🏯🌺💻💑👞🌺😜🏯🎉📭🌺👞🔲🔝🐱👱🌺😜📭📹🎉🏯🔄🏯🔝👤💑💟🌺🕖🌺😜🔄🐱📂👤💑💟🌺👤🏯🌺💻💑👞🌺🔝🎉💡😡🏯👱🌺😜🏯🎉📭🌺👞🔲🔝🐱👱🌺👤📹😜📭🏯🙆💖🕖💟🌺🍊🌺🍊🏯📭🏯🎉🏯🔄💡🌺🏯👤🕖🌺🌙😜🐱🌺🎿🎉🕖👤📹🔝👞🐱🌼📹👞🕖🌺🌙🌺🍊🏯📭🏯🎉🏯🔄🌺👤📹🔝🏯🌺💻💑📭💷🌺😡👞📹🌙👤🏯🐱🌺🍭👞🐱😡📹👤📭👤💑🔄🌺🍊🎉📹😜🕖🌙💑🔄🌺🌙🐱👞🕖🍊🏯🔝💡📂👤💑🔄🌺😜🔄🐱👞💑🔄🌺🌙🐱😜🐱👞💑🔄🌺🏯📭🔝📹🌙📹📭💷😜🙆🌺🌙😜🙆🍊🏯👱🌺😜📭🎉📹😜📭🕖🌺👤🐱🌺🍊🎉📹😜👤🐱🙆🌺🕖🌺👤📹🔝🌺🌙😜🐱🔄🌺🏯😜📭📹👞💷👤💑🔄🌺😜🔄🐱🙆📭💷😜🙆'

In [13]:
decoding_proposals = decode_text(encoded_text, unigram_stat, n_gram, beam_search_width=10)

Beam search:   0%|          | 0/847 [00:00<?, ?it/s]

In [14]:
decoding_proposals[0].phrase

'сроенятг нвублв н укосовьнистам эиои санавдг вапаи юоробаеьяог ыаехтед покклятслч аг клслв повубуивтсда носаид т соозха иоишлн ыа пртечв нсог пртсдшедг иое озрлхаетч н эиойо рокл ыаехтелмт с айо паиарзурйняом мтра сна вжкт рлцкавчвтнь ел ксл носарбаеео проитсоповоыеда норил окте етцбтг нори побвда йвупда т йвлсеоа нмабеда вжкт яоиорда саружи с ио шио океому муыу елко ытиь н океож ыаеог н яоиорож ое озсаешле шио касубяа елко здиь еастееож ыаехтеа нидквтсож муыштеа муыанисаеедм соцкарыедм т исаркдм шио елко сонптидслиь каиаг цлрлзлидслиь нсог юваз пвлитиь ковйт т рлцеда иому покозеда йвупонит эио здв нори вжкаг нилромокедю т нмабедю ео здв круйог нори вжкаг елниочхтю я яоиорому оет сна пртелкваылвт с яоиором елко здиь йвлсеоа эвайлеиедм ярлнтсдм савтяокубедм нмавдм санавдм оиклслиьнч снчяог нирлнит еа ярлнеач т елк снам онилвьедм нмачиьнч'

In [15]:
display(HTML(tabulate([[original_processed, decoding_proposals[0].phrase]], headers=["Оригинальный текст", "Перевод"], stralign="left",  tablefmt='html')))

Оригинальный текст,Перевод
вронский слушал с удовольствием этот веселый лепет хорошенькой женщины поддакивая ей давал полушутливые советы и вообще тотчас же принял свой привычный тон обращения с этого рода женщинами в его петербургском мире все люди разделялись на два совершенно противоположные сорта один низший сорт пошлые глупые и главное смешные люди которые веруют в то что одному мужу надо жить с одною женой с которою он обвенчан что девушке надо быть невинною женщине стыдливою мужчине мужественным воздержным и твердым что надо воспитывать детей зарабатывать свой хлеб платить долги и разные тому подобные глупости это был сорт людей старомодных и смешных но был другой сорт людей настоящих к которому они все принадлежали в котором надо быть главное элегантным красивым великодушным смелым веселым отдаваться всякой страсти не краснея и над всем остальным смеяться,сроенятг нвублв н укосовьнистам эиои санавдг вапаи юоробаеьяог ыаехтед покклятслч аг клслв повубуивтсда носаид т соозха иоишлн ыа пртечв нсог пртсдшедг иое озрлхаетч н эиойо рокл ыаехтелмт с айо паиарзурйняом мтра сна вжкт рлцкавчвтнь ел ксл носарбаеео проитсоповоыеда норил окте етцбтг нори побвда йвупда т йвлсеоа нмабеда вжкт яоиорда саружи с ио шио океому муыу елко ытиь н океож ыаеог н яоиорож ое озсаешле шио касубяа елко здиь еастееож ыаехтеа нидквтсож муыштеа муыанисаеедм соцкарыедм т исаркдм шио елко сонптидслиь каиаг цлрлзлидслиь нсог юваз пвлитиь ковйт т рлцеда иому покозеда йвупонит эио здв нори вжкаг нилромокедю т нмабедю ео здв круйог нори вжкаг елниочхтю я яоиорому оет сна пртелкваылвт с яоиором елко здиь йвлсеоа эвайлеиедм ярлнтсдм савтяокубедм нмавдм санавдм оиклслиьнч снчяог нирлнит еа ярлнеач т елк снам онилвьедм нмачиьнч


# Частотное декодирование на основе частот биграмм

In [16]:
n_gram = 2
source_freq = Counter()
for name in texts:
    if name == "AnnaKarenina.txt":
        big_text = texts[name]
    else:
        source_freq.update(count_ngram(texts[name], n_gram=n_gram))

In [17]:
bigram_stat = NGramStat(source_freq)

In [18]:
encoded_text, original_processed = encode_text(test_text, symmetric_key, n_gram=n_gram)

Encoded text length: 848


In [19]:
encoded_text

'🌙🎉🏯👤😜🍊🕖👱🌺😜👞💡📂📹👞🌺😜🌺💡🔝🏯🌙🏯👞💷😜📭🌙🕖🐱🔄🌺🍭📭🏯📭🌺🌙🐱😜🐱👞💑👱🌺👞🐱🎿🐱📭🌺💟🏯🎉🏯📂🐱👤💷🍊🏯👱🌺🌼🐱👤💖🕖👤💑🌺🎿🏯🔝🔝📹🍊🕖🌙📹🙆🌺🐱👱🌺🔝📹🌙📹👞🌺🎿🏯👞💡📂💡📭👞🕖🌙💑🐱🌺😜🏯🌙🐱📭💑🌺🕖🌺🌙🏯🏯💻💖🐱🌺📭🏯📭👨📹😜🌺🌼🐱🌺🎿🎉🕖👤🙆👞🌺😜🌙🏯👱🌺🎿🎉🕖🌙💑👨👤💑👱🌺📭🏯👤🌺🏯💻🎉📹💖🐱👤🕖🙆🌺😜🌺🍭📭🏯😡🏯🌺🎉🏯🔝📹🌺🌼🐱👤💖🕖👤📹🔄🕖🌺🌙🌺🐱😡🏯🌺🎿🐱📭🐱🎉💻💡🎉😡😜🍊🏯🔄🌺🔄🕖🎉🐱🌺🌙😜🐱🌺👞🔲🔝🕖🌺🎉📹🔫🔝🐱👞🙆👞🕖😜💷🌺👤📹🌺🔝🌙📹🌺😜🏯🌙🐱🎉📂🐱👤👤🏯🌺🎿🎉🏯📭🕖🌙🏯🎿🏯👞🏯🌼👤💑🐱🌺😜🏯🎉📭📹🌺🏯🔝🕖👤🌺👤🕖🔫📂🕖👱🌺😜🏯🎉📭🌺🎿🏯📂👞💑🐱🌺😡👞💡🎿💑🐱🌺🕖🌺😡👞📹🌙👤🏯🐱🌺😜🔄🐱📂👤💑🐱🌺👞🔲🔝🕖🌺🍊🏯📭🏯🎉💑🐱🌺🌙🐱🎉💡🔲📭🌺🌙🌺📭🏯🌺👨📭🏯🌺🏯🔝👤🏯🔄💡🌺🔄💡🌼💡🌺👤📹🔝🏯🌺🌼🕖📭💷🌺😜🌺🏯🔝👤🏯🔲🌺🌼🐱👤🏯👱🌺😜🌺🍊🏯📭🏯🎉🏯🔲🌺🏯👤🌺🏯💻🌙🐱👤👨📹👤🌺👨📭🏯🌺🔝🐱🌙💡📂🍊🐱🌺👤📹🔝🏯🌺💻💑📭💷🌺👤🐱🌙🕖👤👤🏯🔲🌺🌼🐱👤💖🕖👤🐱🌺😜📭💑🔝👞🕖🌙🏯🔲🌺🔄💡🌼👨🕖👤🐱🌺🔄💡🌼🐱😜📭🌙🐱👤👤💑🔄🌺🌙🏯🔫🔝🐱🎉🌼👤💑🔄🌺🕖🌺📭🌙🐱🎉🔝💑🔄🌺👨📭🏯🌺👤📹🔝🏯🌺🌙🏯😜🎿🕖📭💑🌙📹📭💷🌺🔝🐱📭🐱👱🌺🔫📹🎉📹💻📹📭💑🌙📹📭💷🌺😜🌙🏯👱🌺💟👞🐱💻🌺🎿👞📹📭🕖📭💷🌺🔝🏯👞😡🕖🌺🕖🌺🎉📹🔫👤💑🐱🌺📭🏯🔄💡🌺🎿🏯🔝🏯💻👤💑🐱🌺😡👞💡🎿🏯😜📭🕖🌺🍭📭🏯🌺💻💑👞🌺😜🏯🎉📭🌺👞🔲🔝🐱👱🌺😜📭📹🎉🏯🔄🏯🔝👤💑💟🌺🕖🌺😜🔄🐱📂👤💑💟🌺👤🏯🌺💻💑👞🌺🔝🎉💡😡🏯👱🌺😜🏯🎉📭🌺👞🔲🔝🐱👱🌺👤📹😜📭🏯🙆💖🕖💟🌺🍊🌺🍊🏯📭🏯🎉🏯🔄💡🌺🏯👤🕖🌺🌙😜🐱🌺🎿🎉🕖👤📹🔝👞🐱🌼📹👞🕖🌺🌙🌺🍊🏯📭🏯🎉🏯🔄🌺👤📹🔝🏯🌺💻💑📭💷🌺😡👞📹🌙👤🏯🐱🌺🍭👞🐱😡📹👤📭👤💑🔄🌺🍊🎉📹😜🕖🌙💑🔄🌺🌙🐱👞🕖🍊🏯🔝💡📂👤💑🔄🌺😜🔄🐱👞💑🔄🌺🌙🐱😜🐱👞💑🔄🌺🏯📭🔝📹🌙📹📭💷😜🙆🌺🌙😜🙆🍊🏯👱🌺😜📭🎉📹😜📭🕖🌺👤🐱🌺🍊🎉📹😜👤🐱🙆🌺🕖🌺👤📹🔝🌺🌙😜🐱🔄🌺🏯😜📭📹👞💷👤💑🔄🌺😜🔄🐱🙆📭💷😜🙆'

In [20]:
decoding_proposals = decode_text(encoded_text, bigram_stat, n_gram, beam_search_width=20)

Beam search:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
table_data = []
for serach_res in decoding_proposals:
    table_data.append([serach_res.log_likelihood, original_processed, serach_res.phrase])

In [22]:
display(HTML(tabulate(table_data, headers=["Логаримф правдопожобия", "Оригинальный текст", "Перевод"], numalign="left", tablefmt='html')))

Логаримф правдопожобия,Оригинальный текст,Перевод
-825.286,вронский слушал с удовольствием этот веселый лепет хорошенькой женщины поддакивая ей давал полушутливые советы и вообще тотчас же принял свой привычный тон обращения с этого рода женщинами в его петербургском мире все люди разделялись на два совершенно противоположные сорта один низший сорт пошлые глупые и главное смешные люди которые веруют в то что одному мужу надо жить с одною женой с которою он обвенчан что девушке надо быть невинною женщине стыдливою мужчине мужественным воздержным и твердым что надо воспитывать детей зарабатывать свой хлеб платить долги и разные тому подобные глупости это был сорт людей старомодных и смешных но был другой сорт людей настоящих к которому они все принадлежали в котором надо быть главное элегантным красивым великодушным смелым веселым отдаваться всякой страсти не краснея и над всем остальным смеяться,уюпеётетбоъяф цуо шлимьсчеакакгоэтмуенут о и велциалф вдогрсяблдой коанлеалылнощеан сл санься инмоятчтмибыфешкенвальчтм умжнелчтковоя бочаё ковочукаё м анасу хоосщепеэт треы алелдой неоленситоенцужддрф ётс тоспчт пчтдикиузыеа омору нтеантемибыовдолиены зьчаъюсев у тои отелеронзашаетбои маъюыму т ъяывчтцефиа личтвиа качтдикиеммуф у енждатматом енм елери рашиютязваревожабоелеринблдосябоеммуф н санасбыотноннм есгля кязвареи жа н иролих ъеотрочтнаодлочах шикнрочтшиъенабыосщиеназа ивкагоцеакжд бгопоренестенегдотьножа сцуднпуые нниеннию олся двебуатниотю стшецецее цау омс раъюстску т ъяъенацеэтреи цумицобиже нбогах керня кболапрняязи и цемнютсябои мадие ё нена сй ь мея м ы шийстре таановнеалыш сценея м ы гонест идую фиа личтфресны нщиеме шлятгобымой елпрщибои мегобыжамегомуалножавые выя ё нае нацен еме днгушкенвае тагоена сткаговиазнавы
-825.29,вронский слушал с удовольствием этот веселый лепет хорошенькой женщины поддакивая ей давал полушутливые советы и вообще тотчас же принял свой привычный тон обращения с этого рода женщинами в его петербургском мире все люди разделялись на два совершенно противоположные сорта один низший сорт пошлые глупые и главное смешные люди которые веруют в то что одному мужу надо жить с одною женой с которою он обвенчан что девушке надо быть невинною женщине стыдливою мужчине мужественным воздержным и твердым что надо воспитывать детей зарабатывать свой хлеб платить долги и разные тому подобные глупости это был сорт людей старомодных и смешных но был другой сорт людей настоящих к которому они все принадлежали в котором надо быть главное элегантным красивым великодушным смелым веселым отдаваться всякой страсти не краснея и над всем остальным смеяться,уюпеётетбоъяф цуо шлимьсчеакакгоэтмуенут о и велциалф вдогрсяблдой коанлеалылнощеан сл санься инмоятчтмибыфешкенвальчтм умжнелчтковоя бочаё ковочукаё м анасу хоосщепеэт треы алелдой неоленситоенцужддрф ётс тоспчт пчтдикиузыеа омору нтеантемибыовдолиены зьчаъюсев у тои отелеронзашаетбои маъюыму т ъяывчтцефиа личтвиа качтдикиеммуф у енждатматом енм елери рашиютязваревожабоелеринблдосябоеммуф н санасбыотноннм есгля кязвареи жа н иролих ъеотрочтнаодлочах шикнрочтшиъенабыосщиеназа ивкагоцеакжд бгопоренестенегдотьножа сцуднпуые нниеннию олся двебаатниотю стшецецее цау омс раъюстску т ъяъенацеэтреи цумицобиже нбогах керня кболапрняязи и цемнютсябои мадие ё нена сй ь мея м ы шийстре таановнеалыш сценея м ы гонест идую фиа личтфресны нщиеме шлятгобымой елпрщибои мегобыжамегомуалножавые выя ё нае нацен еме днгушкенвае тагоена сткаговиазнавы
-825.293,вронский слушал с удовольствием этот веселый лепет хорошенькой женщины поддакивая ей давал полушутливые советы и вообще тотчас же принял свой привычный тон обращения с этого рода женщинами в его петербургском мире все люди разделялись на два совершенно противоположные сорта один низший сорт пошлые глупые и главное смешные люди которые веруют в то что одному мужу надо жить с одною женой с которою он обвенчан что девушке надо быть невинною женщине стыдливою мужчине мужественным воздержным и твердым что надо воспитывать детей зарабатыват

# Декодирование с помощью MCMC

In [23]:
# test_text = big_text[:1600]

In [24]:
enc_mapping = symmetric_key.get_encode_mapping()

In [25]:
encoded = "".join(map(lambda x: enc_mapping[x], original_processed))

In [26]:
original_processed

'вронский слушал с удовольствием этот веселый лепет хорошенькой женщины поддакивая ей давал полушутливые советы и вообще тотчас же принял свой привычный тон обращения с этого рода женщинами в его петербургском мире все люди разделялись на два совершенно противоположные сорта один низший сорт пошлые глупые и главное смешные люди которые веруют в то что одному мужу надо жить с одною женой с которою он обвенчан что девушке надо быть невинною женщине стыдливою мужчине мужественным воздержным и твердым что надо воспитывать детей зарабатывать свой хлеб платить долги и разные тому подобные глупости это был сорт людей старомодных и смешных но был другой сорт людей настоящих к которому они все принадлежали в котором надо быть главное элегантным красивым великодушным смелым веселым отдаваться всякой страсти не краснея и над всем остальным смеяться'

In [27]:
encoded

'🌙🎉🏯👤😜🍊🕖👱🌺😜👞💡📂📹👞🌺😜🌺💡🔝🏯🌙🏯👞💷😜📭🌙🕖🐱🔄🌺🍭📭🏯📭🌺🌙🐱😜🐱👞💑👱🌺👞🐱🎿🐱📭🌺💟🏯🎉🏯📂🐱👤💷🍊🏯👱🌺🌼🐱👤💖🕖👤💑🌺🎿🏯🔝🔝📹🍊🕖🌙📹🙆🌺🐱👱🌺🔝📹🌙📹👞🌺🎿🏯👞💡📂💡📭👞🕖🌙💑🐱🌺😜🏯🌙🐱📭💑🌺🕖🌺🌙🏯🏯💻💖🐱🌺📭🏯📭👨📹😜🌺🌼🐱🌺🎿🎉🕖👤🙆👞🌺😜🌙🏯👱🌺🎿🎉🕖🌙💑👨👤💑👱🌺📭🏯👤🌺🏯💻🎉📹💖🐱👤🕖🙆🌺😜🌺🍭📭🏯😡🏯🌺🎉🏯🔝📹🌺🌼🐱👤💖🕖👤📹🔄🕖🌺🌙🌺🐱😡🏯🌺🎿🐱📭🐱🎉💻💡🎉😡😜🍊🏯🔄🌺🔄🕖🎉🐱🌺🌙😜🐱🌺👞🔲🔝🕖🌺🎉📹🔫🔝🐱👞🙆👞🕖😜💷🌺👤📹🌺🔝🌙📹🌺😜🏯🌙🐱🎉📂🐱👤👤🏯🌺🎿🎉🏯📭🕖🌙🏯🎿🏯👞🏯🌼👤💑🐱🌺😜🏯🎉📭📹🌺🏯🔝🕖👤🌺👤🕖🔫📂🕖👱🌺😜🏯🎉📭🌺🎿🏯📂👞💑🐱🌺😡👞💡🎿💑🐱🌺🕖🌺😡👞📹🌙👤🏯🐱🌺😜🔄🐱📂👤💑🐱🌺👞🔲🔝🕖🌺🍊🏯📭🏯🎉💑🐱🌺🌙🐱🎉💡🔲📭🌺🌙🌺📭🏯🌺👨📭🏯🌺🏯🔝👤🏯🔄💡🌺🔄💡🌼💡🌺👤📹🔝🏯🌺🌼🕖📭💷🌺😜🌺🏯🔝👤🏯🔲🌺🌼🐱👤🏯👱🌺😜🌺🍊🏯📭🏯🎉🏯🔲🌺🏯👤🌺🏯💻🌙🐱👤👨📹👤🌺👨📭🏯🌺🔝🐱🌙💡📂🍊🐱🌺👤📹🔝🏯🌺💻💑📭💷🌺👤🐱🌙🕖👤👤🏯🔲🌺🌼🐱👤💖🕖👤🐱🌺😜📭💑🔝👞🕖🌙🏯🔲🌺🔄💡🌼👨🕖👤🐱🌺🔄💡🌼🐱😜📭🌙🐱👤👤💑🔄🌺🌙🏯🔫🔝🐱🎉🌼👤💑🔄🌺🕖🌺📭🌙🐱🎉🔝💑🔄🌺👨📭🏯🌺👤📹🔝🏯🌺🌙🏯😜🎿🕖📭💑🌙📹📭💷🌺🔝🐱📭🐱👱🌺🔫📹🎉📹💻📹📭💑🌙📹📭💷🌺😜🌙🏯👱🌺💟👞🐱💻🌺🎿👞📹📭🕖📭💷🌺🔝🏯👞😡🕖🌺🕖🌺🎉📹🔫👤💑🐱🌺📭🏯🔄💡🌺🎿🏯🔝🏯💻👤💑🐱🌺😡👞💡🎿🏯😜📭🕖🌺🍭📭🏯🌺💻💑👞🌺😜🏯🎉📭🌺👞🔲🔝🐱👱🌺😜📭📹🎉🏯🔄🏯🔝👤💑💟🌺🕖🌺😜🔄🐱📂👤💑💟🌺👤🏯🌺💻💑👞🌺🔝🎉💡😡🏯👱🌺😜🏯🎉📭🌺👞🔲🔝🐱👱🌺👤📹😜📭🏯🙆💖🕖💟🌺🍊🌺🍊🏯📭🏯🎉🏯🔄💡🌺🏯👤🕖🌺🌙😜🐱🌺🎿🎉🕖👤📹🔝👞🐱🌼📹👞🕖🌺🌙🌺🍊🏯📭🏯🎉🏯🔄🌺👤📹🔝🏯🌺💻💑📭💷🌺😡👞📹🌙👤🏯🐱🌺🍭👞🐱😡📹👤📭👤💑🔄🌺🍊🎉📹😜🕖🌙💑🔄🌺🌙🐱👞🕖🍊🏯🔝💡📂👤💑🔄🌺😜🔄🐱👞💑🔄🌺🌙🐱😜🐱👞💑🔄🌺🏯📭🔝📹🌙📹📭💷😜🙆🌺🌙😜🙆🍊🏯👱🌺😜📭🎉📹😜📭🕖🌺👤🐱🌺🍊🎉📹😜👤🐱🙆🌺🕖🌺👤📹🔝🌺🌙😜🐱🔄🌺🏯😜📭📹👞💷👤💑🔄🌺😜🔄🐱🙆📭💷😜🙆'

In [28]:
for i in range(2):
    print(mcmc_decryption(encoded, bigram_stat, symmetric_key.src_vocab(), symmetric_key.chiper_vocab(), symmetric_key, 10_000)[0], "\n")

  0%|          | 0/10000 [00:00<?, ?it/s]

тд лряспернубанереук т нырвтсомещв веторонипенойовец д болыя пегольслией ккаястажеопекатаней нубувнстиоер товиесет  шьоев взарегоейдслжнерт пейдстизлипев ле шдаьолсжерещв х ед каегольсламсетеох ейоводшудхря мемсдоетроенчкседаюконжнсрыелаектаер тодболл ейд вст й н глиоер двае кслелсюбспер двей бниоехнуйиоесехнатл оермоблиоенчксея в диоетодучветев езв е кл муемугуелак егсвыере кл чегол перея в д че ле штолзалезв екотубяоелак ешивыелотслл чегольслоервикнст чемугзслоемугорвтоллимет юкодглимесевтодкимезв елак ет рйсвитавыековопеюадашавитавыерт пецношейнавсвыек нхсеседаюлиоев муей к шлиоехнуй рвсещв ешинер двенчкопервад м клицесермоблицел ешинекдух пер двенчкопеларв жьсцеяея в д муе лсетроейдслакногансетея в д мелак ешивыехнатл оещнохалвлимеядарстиметонся кублимермониметорониме вкатавыржетржя первдарвселоеядарложеселакетроме рванылимерможвырж 



  0%|          | 0/10000 [00:00<?, ?it/s]

вронский слушал с удовольствием этот веселый лепет хорошенькой женщины поддакивая ей давал полушутливые советы и вообще тотчас же принял свой привычный тон обращения с этого рода женщинами в его петербургском мире все люди разделялись на два совершенно противоположные сорта один низший сорт пошлые глупые и главное смешные люди которые веруют в то что одному мужу надо жить с одною женой с которою он обвенчан что девушке надо быть невинною женщине стыдливою мужчине мужественным воздержным и твердым что надо воспитывать детей зарабатывать свой хлеб платить долги и разные тому подобные глупости это был сорт людей старомодных и смешных но был другой сорт людей настоящих к которому они все принадлежали в котором надо быть главное элегантным красивым великодушным смелым веселым отдаваться всякой страсти не краснея и над всем остальным смеяться 



In [29]:
test = "დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ"

In [30]:
for char in test:
    assert len(char) == 1

In [31]:
len(set(test))

28

In [32]:
len(string.ascii_lowercase)

26